In [1]:
import matplotlib.pyplot as plt
import numpy as np
from   tensorflow.keras import regularizers
from   tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from   tensorflow.keras.datasets import fashion_mnist
from   tensorflow.keras.layers import Dense, Input
from   tensorflow.keras.models import Model

%matplotlib inline

In [2]:
(X_train, _), (X_test, _) = fashion_mnist.load_data()
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [3]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [4]:
# Flatten
X_train = X_train.reshape((X_train.shape[0], np.prod(X_train.shape[1:])))
X_test = X_test.reshape((X_test.shape[0], np.prod(X_test.shape[1:])))

In [5]:
X_train.shape, X_test.shape

((60000, 784), (10000, 784))

In [6]:
image_dim = X_train.shape[1]
encoding_dim = 32

In [7]:
input_image = Input(shape=(image_dim,))
encoded_image = Dense(
    encoding_dim, 
    activation='relu', 
    activity_regularizer=regularizers.l1(10e-5)
)(input_image)
encoder = Model(input_image, encoded_image)
decoded_image = Dense(image_dim, activation='sigmoid')(encoded_image)
autoencoder = Model(input_image, decoded_image)

In [8]:
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [9]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [10]:
checkpointer1 = ModelCheckpoint(
    filepath='modWBest.hdf5', verbose=2, save_best_only=True)

In [11]:
checkpointer2 = EarlyStopping(monitor='val_loss', 
                              min_delta=0.0005, 
                              patience=2, 
                              verbose=2, 
                              mode='auto')

In [12]:
EPOCHS = 50
BATCH = 256
autoencoder.fit(X_train, 
                X_train, 
                epochs=EPOCHS, 
                batch_size=BATCH, 
                verbose=2, 
                shuffle=True, 
                validation_data=(X_test, X_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 - 4s - loss: 0.6942 - val_loss: 0.6941
Epoch 2/50
60000/60000 - 3s - loss: 0.6939 - val_loss: 0.6938
Epoch 3/50
60000/60000 - 3s - loss: 0.6936 - val_loss: 0.6935
Epoch 4/50
60000/60000 - 3s - loss: 0.6933 - val_loss: 0.6932
Epoch 5/50
60000/60000 - 3s - loss: 0.6931 - val_loss: 0.6929
Epoch 6/50
60000/60000 - 3s - loss: 0.6928 - val_loss: 0.6927
Epoch 7/50
60000/60000 - 3s - loss: 0.6925 - val_loss: 0.6924
Epoch 8/50
60000/60000 - 3s - loss: 0.6923 - val_loss: 0.6922
Epoch 9/50
60000/60000 - 3s - loss: 0.6920 - val_loss: 0.6919
Epoch 10/50
60000/60000 - 3s - loss: 0.6918 - val_loss: 0.6917
Epoch 11/50
60000/60000 - 3s - loss: 0.6915 - val_loss: 0.6914
Epoch 12/50
60000/60000 - 3s - loss: 0.6913 - val_loss: 0.6912
Epoch 13/50
60000/60000 - 3s - loss: 0.6910 - val_loss: 0.6909
Epoch 14/50
60000/60000 - 3s - loss: 0.6908 - val_loss: 0.6907
Epoch 15/50
60000/60000 - 3s - loss: 0.6906 - val_loss: 0.6905
Epoch 16/50
60

In [13]:
encoded_images = encoder.predict(X_test)         # compress
decoded_images = decoder.predict(encoded_images) # decompress

In [14]:
# With checkpoints and saving:
'''
autoencoder.fit(X_train,
                X_train,
                epochs=EPOCHS,
                batch_size=BATCH,
                verbose=2,
                callbacks=[checkpointer1], # or checkpointer2
                shuffle=True,
                validation_data=(X_test, X_test))
'''

'\nautoencoder.fit(X_train,\n                X_train,\n                epochs=EPOCHS,\n                batch_size=BATCH,\n                verbose=2,\n                callbacks=[checkpointer1], # or checkpointer2\n                shuffle=True,\n                validation_data=(X_test, X_test))\n'

In [15]:
#autoencoder.load_weights('modWBest.hdf5')
#encoded_images = encoder.predict(X_test)
#decoded_images = decoder.predict(encoded_images)

In [ ]:
n_items = 12
plt.figure(figsize=(20, 4))

for i in range(n_items):
    # original:
    graph = plt.subpot(2, n_items, i + 1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    graph.get_xaxis().set_visible(False)
    graph.get_yaxis().set_visible(False)
    
    # compressed/decompressed
    graph = plt.subplot(2, n_items, i + 1 + n_items)
    plt.imshow(decoded_images[i].reshape(28, 28))
    plt.gray()
    graph.get_xaxis().set_visible(False)
    graph.get_yaxis().set_visible(False)